# CS4035 - Cyber Data Analytics
## Lab 3 

### Submit on brightspace (zip file with the name Group_xx.zip)
(i) This jupyter file completed with code, plots, figures and report for each question. Additional plots and figures can be created for explanation before the end of each main question. Lab 3 contains 4 main questions + 1 bonus. Write the code or explanation below each sub question. For the explantions, include what you would normally include in the report for this lab assignment, for example data pre-processing, hypothesis tested, approach, results, etc.
(ii) A PDF or a Word report for the assignment. Create a report from the plots, figures, tables and the write-up that you provide in this jupyter file. The report will be used as a proof for page limit. 
(iii) The libraries needed to run this file. 

Your peers should be able to use the readme section for instructions and be able to run this file. 

## Group Number : 42

## Student 1 
### Name : Otte van Dam
### ID : 5096790

## Student 2
### Name : Suhaib Basir
### ID : 5059151

## Readme - Provide instructions - libraries used, location of the data file, etc. Keep it short. Remember your peers will not debug your code and should be able to reproduce the exact output you provide.

In [2]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nbformat
from tqdm import tqdm

nb = nbformat.read("Lab3.ipynb", nbformat.NO_CONVERT)
word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))

print("Word count:", word_count)

Word count: 326


## 1. Familiarization and discretization task – 1 A4 (5 Points)

In [32]:
# Specify the columns you want to read as strings
string_columns = ['sTos', 'dTos']

data = "dataset_lab3/dataset_10/capture20110818.binetflow"
df1 = pd.read_csv(data, dtype={col: int for col in string_columns})
df1['sTos'].fillna(0)


ValueError: Integer column has NA values in column 9

In [18]:
# Use the value_counts() function to count the occurrences of each value in the 'SrcAddr' column
srcaddr_counts = df1['SrcAddr'].value_counts()

# Get the most occurring value by accessing the first element of the resulting series
most_common_value = srcaddr_counts.index[0]

# Print the most occurring value
print("Most occurring value in 'SrcAddr':", most_common_value, ' with ', srcaddr_counts[0], ' occurences, out of ', len(df1), ' occurences in total')

# Discretize numeric columns using percentiles
for column in df1.columns:
    if df1[column].dtype == np.float64 or df1[column].dtype == np.int64:
        df1[column] = pd.qcut(df1[column], q=10, labels=False, duplicates='drop')

Most occurring value in 'SrcAddr': 147.32.84.138  with  211967  occurences, out of  1309791  occurences in total
                          StartTime  Dur Proto          SrcAddr  Sport    Dir  \
0        2011/08/18 10:21:46.633335    7   tcp     93.45.239.29   1611     ->   
1        2011/08/18 10:19:49.027650    8   tcp   62.240.166.118   1031    <?>   
2        2011/08/18 10:22:07.160628    8   tcp    147.32.86.148  58067     ->   
3        2011/08/18 10:26:02.052163    7   tcp      147.32.3.51   3130     ->   
4        2011/08/18 10:26:52.226748    7   tcp    88.212.37.169   3134     ->   
...                             ...  ...   ...              ...    ...    ...   
1309786  2011/08/18 15:04:59.579762    1   tcp  219.129.213.146  25463     ->   
1309787  2011/08/18 15:04:59.626719    4   udp  213.220.104.152  64126    <->   
1309788  2011/08/18 15:04:59.686325    4   udp     59.22.11.248  44432    <->   
1309789  2011/08/18 15:04:59.690632    0   udp    95.102.170.66  16978     ->

### 1a. Plot visualizations - Select and visualize two features for modeling the behavior of the infected host.

### 1b. Discretize selected features

### 1c. Answers and explanation

## 2. Sketching task – 1/2 A4 (Individual, 10 Points)

### 2a. COUNT-MIN sketch


### 2b. Analysis and answers to the questions

## 3. Min-wise locality sensitive hashing task – 1/2 A4 (Individual, 10 Points)

### 3a. Implementation

### 3b. Analysis and answers to the questions

## 4.Botnet profiling and fingerprinting task – 1 A4 (5 Points)

### 4a. Learn a sequential model, profiling and fingerprinting

### 4b. Analysis and answers to the questions.

## 5. Bonus Task 1/2 A4 (5 Points)

### 5a. Provide implementation and analysis.